In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import preprocessing
from vanilla_rnn import RNN
from dio import load_data
import numpy as np
import joblib

In [ ]:
snippet = load_data('../data/silmarillion.txt')

The data set consists of a list of lines which tells the story of the high elves struggle against Morgoth and his forces.

In [ ]:
snippet[:3]

array(['AINULINDALË', 'The Music of the Ainur',
       'There was Eru, the One, who in Arda is called Ilúvatar; and he made first the Ainur, the Holy Ones, that were the offspring of his thought, and they were with him before aught else was made. And he spoke to them, propounding to them themes of music; and they sang before him, and he was glad. But for a long while they sang only each alone, or but few together, while the rest hearkened; for each comprehended only that part of me mind of Ilúvatar from which he came, and in the understanding of their brethren they grew but slowly. Yet ever as they listened they came to deeper understanding, and increased in unison and harmony.'],
      dtype='<U2728')

In [ ]:
vocabulary = preprocessing.create_vocabulary(snippet)

In [ ]:
vocabulary['vocab_size']

93

The vocab size here is relatively large for a couple of reasons

1. There's a lot of unconvential characters like Ë and ú
2. It has both uppercase and lower case letters.

The first is a feature, the second definitely makes life unecessarily difficult for the model since it has to learn upper case letter separate from lower case letters. It is unlikely that there's enough data here to facilitate that.

The character level RNN model is trained on (input, target) sequences that are constructed out of a sequence of text as follows:

In [ ]:
# Split into train test and validation set while preserving individual lines intact.
input_lines, test_lines = preprocessing.train_test_split(snippet, test_size=0.1)
train_lines, val_lines = preprocessing.train_test_split(input_lines, test_size=0.1)

X_train, y_train = preprocessing.text_to_input_and_target(train_lines)
X_val, y_val = preprocessing.text_to_input_and_target(val_lines)
X_test, y_test = preprocessing.text_to_input_and_target(test_lines)


In [ ]:
print(f"'{X_test[:20]}'", "gets transformed into the following sequence of inputs and outputs: \n")
print([(input, target) for (input, target) in zip(X_test[:20],y_test[:20])])

'Then Turgon answered' gets transformed into the following sequence of inputs and outputs: 

[('T', 'h'), ('h', 'e'), ('e', 'n'), ('n', ' '), (' ', 'T'), ('T', 'u'), ('u', 'r'), ('r', 'g'), ('g', 'o'), ('o', 'n'), ('n', ' '), (' ', 'a'), ('a', 'n'), ('n', 's'), ('s', 'w'), ('w', 'e'), ('e', 'r'), ('r', 'e'), ('e', 'd'), ('d', ':')]


I already trained a model consisting of a single layer RNN with a hidden state size of 200 on the full text of the Silmarillion, various checkpoints are saved in `model_checkpoints`. For now we can just load one and have some fun.

In [ ]:
network = joblib.load('model_checkpoints/model_40_1.393801855652784.joblib')

We can ask the RNN to start inventing some silmarillion like text. The "temperature" parameter used to sample text is quite important - at $\beta = 1$ the text is more or less gibberish, although it must be said that it's very Tolkein-esque gibberish.  

In [ ]:
''.join(network.sample(" ", 1, 200))

' aw-bather darkn; so alwës whithe wes rewelled himselves crembled Sirianuë, last that as they will heast entumened. Finding had to many his goberess to dent. Therefore listen the stor things and not co'

At lower temperatures (higher $\beta$) the model is more conservative and it produces a sequence consisting mostly of actual words. I like the part about how the house of Indwalt became the house of Indwald. 

In [ ]:
''.join(network.sample(" ", 3, 200))

' hen as he was greater than the march of the house of Indwalt, and became the house of Indwald the fear and sought to long became from the marner of the Elves and the crain of the Elves and the Valar t'

We can also prime the RNN by feeding it a sequence of text to consume. This has the effect of setting up a specific hidden state prior to starting sampling. Again, the temperature is an important parameter in generating samples from the RNN.

In [ ]:
prompt, _ = preprocessing.text_to_input_and_target(load_data('sample_input.txt'))

In [ ]:
prompt

'But Morgoth thought that his triumph was fulfilled, recking little of the sons of Feanor, and of their oath, which had harmed him never and turned always to his mightiest aid; and in his black thought he laughed, regretting not the one Silmaril that he had lost, for by it as he deemed the last shred of the people of the Eldar should vanish from middle earth and trouble it no more'

In [ ]:
''.join(network.sample(prompt, 1, 200))

'ed that was himself, and in the wooth tume they were foundainst foreshe. And he was have made of itsolf departed, in the under the thith they we their raison were unto was the sonr-anciodse were willin'

In [ ]:
''.join(network.sample(prompt, 3, 200))

'ed the fair was near to the house of Andwolle, and the stars of the Noldor, and they were become the masters of the Elves and the marches of the west of the Elves and Men should be many perils and the '

Hey that second generated line looks pretty reasonable! The RNN knows about the Noldor, love of the stars, and their fateful march to the west (with the many perils that followed). 